<a href="https://colab.research.google.com/github/mrrajatgarg/Music_Classification_And_Tagging_/blob/master/script_to_run_2dcnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [0]:
import numpy as np
import pandas as pd

In [0]:
from skimage.io import imread
from skimage.transform import resize

In [0]:
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.utils import to_categorical

import kapre
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D

In [0]:
import sklearn
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

# Downloading of dataset

**X_train data: **

https://drive.google.com/open?id=1FuPakB_kvoLWyFGSrGdiDt-ujNqFX8gH

**y_train_pickle data: **

https://drive.google.com/open?id=1-3I-y1JcjemDs709V7tA2KGR7phHBOMi

**X_val data: **

https://drive.google.com/open?id=1ei40eXUUQBOcGVERXEnaesJtsLjjEGCr

**y_val data: **

https://drive.google.com/open?id=1xxaqhW7tggAY7YYE9UDUu7HsKIsYS6HW

In [0]:
X_train = np.load('path to training data')
y_train_pickle = pd.read_pickle('path to labels of training data')
y_train = y_train_pickle.values
X_train = X_train.reshape(X_train.shape[0], 1366, 96, 1)

In [0]:
print(X_train.shape)
print(y_train.shape)

In [0]:
X_val = np.load('path to validation data')
y_val_pickle = pd.read_pickle('path to labels of validation data')
y_val = y_val_pickle.values
X_val = X_val.reshape(X_val.shape[0], 1366, 96, 1)

In [0]:
print(X_val.shape)
print(y_val.shape)

# Model architecture

In [0]:

class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [0]:
input_shape = X_train.shape[1:]  
print(input_shape)

In [0]:
model = Sequential()
model.add(Melspectrogram(n_dft=1024, n_hop=512, input_shape=input_shape, return_decibel_melgram=True, name='melgram'))
model.add(Normalization2D(str_axis='freq'))

# Layer 1
model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) 

# Layer 2
model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten()) 

model.add(Dense(100, activation='relu')) 
# model.add(BatchNormalization())
model.add(Dense(50,activation='sigmoid'))

model.compile(loss='binary_crossentropy' , optimizer='sgd' , metrics=['accuracy'])

In [0]:
model.summary()

Change the number of epochs as you want
here, epochs = 10

In [0]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), 
          callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_val, y_val))],
         batch_size=32, epochs=100)